In [ ]:
# Define functions to connect to Google and change directories
def connectDrive():
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

def changeDirectory(path):
    import os
    original_path = os.getcwd()
    os.chdir(path)
    new_path = os.getcwd()
    print("Original path: ",original_path)
    print("New path: ",new_path)

# Connect to Google Drive
connectDrive()

# Change path
changeDirectory("/content/drive/My Drive/github/find_p/")

In [ ]:
# !pip install vectorbt
# !pip install pandas_ta
# !pip install yfinance
# !pip install plotly
# !pip install numba
# !apt-get install xvfb libgtk2.0-0 libgconf-2-4
# !wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
# !chmod +x /usr/local/bin/orca

In [1]:
import numpy as np
import vectorbt as vbt
import pandas as pd
from hyperopt import fmin, tpe, hp
from hyperopt.pyll import scope
from datetime import datetime
from pdta_vt_utils import dl, dtmask

# Portfolio Setting

In [2]:
cheight, cwidth = 500, 1000 # Adjust as needed for Chart Height and Width
vbt.settings.set_theme("dark") # Options: "light" (Default), "dark" (my fav), "seaborn"

# Must be set
vbt.settings.portfolio["freq"] = "1D" # Daily

# Predefine vectorbt Portfolio settings
vbt.settings.portfolio["init_cash"] = 1000
vbt.settings.portfolio["fees"] = 0.0025 # 0.25%
vbt.settings.portfolio["slippage"] = 0.0025 # 0.25%
# vbt.settings.portfolio["size"] = 100
# vbt.settings.portfolio["accumulate"] = False
vbt.settings.portfolio["allow_partial"] = False

pf_settings = pd.DataFrame(vbt.settings.portfolio.items(), columns=["Option", "Value"])
pf_settings.set_index("Option", inplace=True)

print(f"Portfolio Settings [Initial]")
pf_settings

Portfolio Settings [Initial]


,Value
Option,
call_seq,default
init_cash,1000
size,inf
size_type,amount
fees,0.0025
fixed_fees,0.0
slippage,0.0025
reject_prob,0.0
min_size,0.0


# Data Collection

In [3]:
asset_tickers = ["AAPL", "TSLA", "TWTR", "SPXL", "^GSPC", "SPY"]

print("="*100)
print("Tickers by index #")
print(f"    Assets: {', '.join([f'{k}: {v}' for k,v in enumerate(asset_tickers)])}")

asset_name = asset_tickers[3] # Change index for different symbol
print("="*100)
print(f"Selected Benchmark | Asset: {asset_name}")
print("="*100)

assets = dl(asset_tickers, lc_cols=True)

start_date = datetime(2010, 1, 1) # Adjust as needed
start_date = pd.to_datetime(start_date).tz_localize('America/New_York')
end_date = datetime(2015, 1, 1)   # Adjust as needed
end_date = pd.to_datetime(end_date).tz_localize('America/New_York')

print("="*100)
print("Available Data:")
print("="*100)
print(f"Assets: {', '.join(assets.keys())}")
print("="*100)

assetdf = assets[asset_name]

# Set True if you want to constrain Data between start_date & end_date
common_range = True
crs = ''
if common_range:
    crs = f" from {start_date} to {end_date}"
    assetdf = dtmask(assetdf, start_date, end_date)

# Update DataFrame names
assetdf.name = asset_name
print("="*100)
print(f"Analysis of:  {assetdf.name}{crs}")
print("="*100)

Tickers by index #
    Assets: 0: AAPL, 1: TSLA, 2: TWTR, 3: SPXL, 4: ^GSPC, 5: SPY
Selected Benchmark | Asset: SPXL
[i] Downloading: AAPL, TSLA, TWTR, SPXL, ^GSPC, SPY
[+] AAPL(10632, 7) Monday February 13, 2023, NYSE: 14:55:16
[+] TSLA(3179, 7) Monday February 13, 2023, NYSE: 14:55:16
[+] TWTR(2259, 7) Monday February 13, 2023, NYSE: 14:55:17
[+] SPXL(3592, 8) Monday February 13, 2023, NYSE: 14:55:17
[+] ^GSPC(23894, 7) Monday February 13, 2023, NYSE: 14:55:18
[+] SPY(7565, 8) Monday February 13, 2023, NYSE: 14:55:18
[*] Download Complete

Available Data:
Assets: AAPL, TSLA, TWTR, SPXL, ^GSPC, SPY
Analysis of:  SPXL from 2010-01-01 00:00:00-05:00 to 2015-01-01 00:00:00-05:00


In [4]:
assetdf

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2010-01-04 00:00:00-05:00,4.100749,4.207282,4.099988,4.200433,28238400,0.0,0.0,0.0
2010-01-05 00:00:00-05:00,4.195866,4.242284,4.147166,4.240001,33206400,0.0,0.0,0.0
2010-01-06 00:00:00-05:00,4.227827,4.278810,4.217935,4.251416,44194800,0.0,0.0,0.0
2010-01-07 00:00:00-05:00,4.229349,4.315335,4.188257,4.301638,43773600,0.0,0.0,0.0
2010-01-08 00:00:00-05:00,4.264352,4.352622,4.240001,4.347295,39685200,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2014-12-24 00:00:00-05:00,21.372377,21.452072,21.266899,21.290339,1291600,0.0,0.0,0.0
2014-12-26 00:00:00-05:00,21.445042,21.606777,21.421602,21.480202,2223200,0.0,0.0,0.0
2014-12-29 00:00:00-05:00,21.400508,21.634906,21.391132,21.559898,2677600,0.0,0.0,0.0


# Indicator Setting Use this section for custome indicators

In [5]:
# def wavetrend(high, low, close, n1, n2):
#     ap = (high + low + close) / 3.0  # HLC3
#     ap = pd.DataFrame(ap)
#     esa = ap.ewm(span=n1, adjust=False).mean()  # EMA
#     d = (abs(ap - esa)).ewm(span=n1, adjust=False).mean()
#     ci = (ap - esa) / (0.015 * d)
#     wt1 = ci.ewm(span=n2, adjust=False).mean()
#     wt2 = wt1.rolling(4).mean()
#     return wt1, wt2
#
#
# Wavetrend = vbt.IndicatorFactory(
#     class_name='Wavetrend',
#     input_names=['high', 'low', 'close'],
#     param_names=['n1', 'n2'],
#     output_names=['wt1', 'wt2'], ).from_apply_func(wavetrend)
#


# Optimization

In [5]:
below = 30,
above = 70


def optimize(params):
    fast, slow, signal, sl_stop = params
    macd_ind = vbt.MACD.run(assetdf['Close'],
                            fast_window=fast,
                            slow_window=slow,
                            signal_window=signal)

    entries = macd_ind.macd_above(0) & macd_ind.macd_above(macd_ind.signal)
    exits = macd_ind.macd_below(0) | macd_ind.macd_below(macd_ind.signal)

    pf = vbt.Portfolio.from_signals(close=assetdf['Close'], entries=entries, exits=exits, sl_stop=sl_stop, direction="longonly")

    print(f"total profit {pf.total_profit()} with parameters: {fast}, {slow}, {signal}, {sl_stop}")
    return np.array([pf.total_profit()])


def optimize_all(params):
    res = optimize(params)
    return sum(res) * -1




best = fmin(optimize_all,
            space=[scope.int(hp.uniform('fast', 2, 51)),
                   scope.int(hp.uniform('slow', 2, 51)),
                   scope.int(hp.uniform('signal', 2, 21)),
                   hp.uniform('sl_stop', 0.01, 0.1)],
            algo=tpe.suggest,
            max_evals=100)

print(best)

total profit 121.4736794553329 with parameters: 49, 9, 5, 0.04827608438893604
total profit 167.02843905019324 with parameters: 16, 38, 18, 0.07976345394651232 
total profit -71.63936929666659 with parameters: 10, 22, 5, 0.09755493554620898  
total profit -19.50592893526664 with parameters: 4, 17, 16, 0.023410984835713883  
total profit -772.4528681808779 with parameters: 4, 25, 2, 0.08023989072281668    
total profit -409.6660395425066 with parameters: 9, 3, 5, 0.037902961819404425    
total profit -311.0673150698682 with parameters: 16, 3, 19, 0.04805624149690825   
total profit 597.3491930701394 with parameters: 39, 35, 2, 0.05392869916475385    
total profit -298.46453717739075 with parameters: 7, 23, 18, 0.03646783760333909  
total profit 77.81315050503156 with parameters: 30, 18, 3, 0.0566346654260877    
total profit -578.4671010610598 with parameters: 24, 23, 11, 0.06527197666652151  
total profit 15.755246165171457 with parameters: 50, 43, 10, 0.0524389204080687   
total profit

In [ ]:
fast = 36 #math.ceil(best['k_window'])
signal = 11 #math.ceil(best['d_window'])
slow = 25 #math.ceil(best['d_ewm'])
sl_stop = best['sl_stop']

In [ ]:
macd_ind = vbt.MACD.run(assetdf['Close'],
                            fast_window=fast,
                            slow_window=slow,
                            signal_window=signal)

entries = macd_ind.macd_above(0) & macd_ind.macd_above(macd_ind.signal)
exits = macd_ind.macd_below(0) | macd_ind.macd_below(macd_ind.signal)

pf = vbt.Portfolio.from_signals(close=assetdf['Close'],
                                entries=entries,
                                exits=exits,
                                sl_stop=sl_stop,
                                direction="longonly")

pf.stats()

In [ ]:
pf.plot().show_png()

In [ ]:
macd_ind.plot().show_png()